In [4]:
import os
import requests
import json
from dotenv import dotenv_values

# Load environment variables
env = dotenv_values("../../../.env")

# Configuration
POLARIS_URL = "http://localhost:8181"
POLARIS_USER = "pyiceberg_service_account"
POLARIS_USER_ROLE = "pyiceberg_service_account_role"
POLARIS_CATALOG_ROLE = "pyiceberg_service_account_catalog_role"
CATALOG_NAME = "pyiceberg_catalog"

In [5]:
env.get("CLIENT_ID")

'035fed64a043451f'

In [6]:
# Get principal token
def get_principal_token():
    response = requests.post(
        f"{POLARIS_URL}/api/catalog/v1/oauth/tokens",
        data={
            "grant_type": "client_credentials",
            "client_id": env.get("CLIENT_ID"),
            "client_secret": env.get("CLIENT_SECRET"),
            "scope": "PRINCIPAL_ROLE:ALL"
        }
    )
    response.raise_for_status()
    return response.json()["access_token"]

In [7]:
# Create catalog
def create_catalog(principal_token):
    headers = {
        "Authorization": f"Bearer {principal_token}",
        "Content-Type": "application/json"
    }
    
    payload = {
            "name": CATALOG_NAME ,
            "type": "INTERNAL",
            "readOnly": False,
            "properties": {
                "default-base-location": "s3://iceberg",
                "s3.endpoint": env.get("AWS_ENDPOINT_URL"),
                "s3.access-key-id": env.get("AWS_ACCESS_KEY_ID"),
                "s3.secret-access-key": env.get("AWS_SECRET_ACCESS_KEY"),
                "s3.path-style-access": "true",
                "s3.region": env.get("AWS_REGION"),
                "s3.allow-http": "true"
            },
            "storageConfigInfo": {
                "storageType": "S3",
                "allowedLocations": ["s3://iceberg"],
                "credentials": {
                    "type": "ACCESS_KEY",
                    "accessKeyId": env.get("AWS_ACCESS_KEY_ID"),
                    "secretAccessKey": env.get("AWS_SECRET_ACCESS_KEY")
                },
                "roleArn": env.get("AWS_ROLE_ARN")
            }
    }
    
    response = requests.post(
        f"{POLARIS_URL}/api/management/v1/catalogs",
        headers=headers,
        json=payload
    )
    response.raise_for_status()
    return response.status_code

In [8]:


# Create principal and get credentials
def create_principal(principal_token):
    headers = {
        "Authorization": f"Bearer {principal_token}",
        "Content-Type": "application/json"
    }
    
    # Create principal
    principal_payload = {
        "name": POLARIS_USER,
        "type": "user"
    }
    
    principal_response = requests.post(
        f"{POLARIS_URL}/api/management/v1/principals",
        headers=headers,
        json=principal_payload
    )
    # principal_response.raise_for_status()
    principal_data = principal_response.json()

    return principal_data

# Create principal role
def create_principal_role(principal_token):
    headers = {
        "Authorization": f"Bearer {principal_token}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "principalRole": {
            "name": POLARIS_USER_ROLE
        }
    }
    
    response = requests.post(
        f"{POLARIS_URL}/api/management/v1/principal-roles",
        headers=headers,
        json=payload
    )
    response.raise_for_status()
    return response.json()

# Assign principal role to user
def assign_principal_role(principal_token):
    headers = {
        "Authorization": f"Bearer {principal_token}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "principalRole": {
            "name": POLARIS_USER_ROLE
        }
    }
    
    response = requests.put(
        f"{POLARIS_URL}/api/management/v1/principals/{POLARIS_USER}/principal-roles",
        headers=headers,
        json=payload
    )
    # response.raise_for_status()
    return response.status_code

# Create catalog role
def create_catalog_role(principal_token):
    headers = {
        "Authorization": f"Bearer {principal_token}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "catalogRole": {
            "name": POLARIS_CATALOG_ROLE
        }
    }
    
    response = requests.post(
        f"{POLARIS_URL}/api/management/v1/catalogs/{CATALOG_NAME}/catalog-roles",
        headers=headers,
        json=payload
    )
    # response.raise_for_status()
    return response.json()

# Assign catalog role to principal role
def assign_catalog_role(principal_token):
    headers = {
        "Authorization": f"Bearer {principal_token}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "catalogRole": {
            "name": POLARIS_CATALOG_ROLE
        }
    }
    
    response = requests.put(
        f"{POLARIS_URL}/api/management/v1/principal-roles/{POLARIS_USER_ROLE}/catalog-roles/{CATALOG_NAME}",
        headers=headers,
        json=payload
    )
    # response.raise_for_status()
    return response

# Grant privileges to catalog role
def grant_privileges(principal_token):
    headers = {
        "Authorization": f"Bearer {principal_token}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "grant": {
            "type": "catalog",
            "privilege": "CATALOG_MANAGE_CONTENT"
        }
    }
    
    response = requests.put(
        f"{POLARIS_URL}/api/management/v1/catalogs/{CATALOG_NAME}/catalog-roles/{POLARIS_CATALOG_ROLE}/grants",
        headers=headers,
        json=payload
    )
    # response.raise_for_status()
    return response



In [9]:
# Main execution
def main():
    try:
        # Get principal token
        print("Getting principal token...")
        principal_token = get_principal_token()
        print(principal_token)
        
        # Create catalog
        print("Creating catalog...")
        create_catalog(principal_token)
        
        # Create principal and get credentials
        print("Creating principal and getting credentials...")
        principal_data = create_principal(principal_token)

        print("principal:", principal_data)
        
        # Create principal role
        print("Creating principal role...")
        create_principal_role(principal_token)
        
        # Assign principal role to user
        print(f"Assigning principal role to {POLARIS_USER}...")
        assign_principal_role(principal_token)
        
        # Create catalog role
        print("Creating catalog role...")
        create_catalog_role(principal_token)
        
        # Assign catalog role to principal role
        print("Assigning catalog role to principal role...")
        assign_catalog_role(principal_token)
        
        # Grant privileges to catalog role
        print("Granting privileges to catalog role...")
        grant_privileges(principal_token)
        
        print("Setup complete!")
        
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response: {e.response.text}")

principal_token = get_principal_token()


In [ ]:
assign_catalog_role(principal_token=principal_token)

In [ ]:
main()

In [10]:
def get_principals(principal_token):
    """Retrieve all principals from Polaris"""
    headers = {
        "Authorization": f"Bearer {principal_token}",
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.get(
            f"{POLARIS_URL}/api/management/v1/principals",
            headers=headers
        )
        
        # Print response status and content for debugging
        print(f"Response status: {response.status_code}")
        print(f"Response content: {response.text}")
        
        # Check if response is empty
        if not response.text:
            print("Error: Empty response from server")
            return []
        
        # Try to parse JSON response
        try:
            principals = response.json()
            return principals
        except json.JSONDecodeError as e:
            print(f"Error: Could not parse JSON response: {e}")
            print(f"Response content: {response.text}")
            return []
            
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return []


In [11]:
get_principals(get_principal_token())

Response status: 200
Response content: {"principals":[{"name":"root","clientId":"035fed64a043451f","properties":{},"createTimestamp":1745151616360,"lastUpdateTimestamp":1745151616360,"entityVersion":1}]}


{'principals': [{'name': 'root',
   'clientId': '035fed64a043451f',
   'properties': {},
   'createTimestamp': 1745151616360,
   'lastUpdateTimestamp': 1745151616360,
   'entityVersion': 1}]}

In [13]:
headers = {
"Authorization": f"Bearer {get_principal_token()}",
"Content-Type": "application/json"
}

response = requests.get(
    f"{POLARIS_URL}/api/management/v1/catalogs/",
    headers=headers
)
response.json()

{'catalogs': [{'type': 'INTERNAL',
   'name': 'quickstart_catalog',
   'properties': {'default-base-location': 's3://iceberg'},
   'createTimestamp': 1745152323735,
   'lastUpdateTimestamp': 1745152323735,
   'entityVersion': 1,
   'storageConfigInfo': {'storageType': 'S3',
    'roleArn': 'arn:aws:iam::123456789012:role/polaris-s3-access',
    'externalId': None,
    'userArn': None,
    'allowedLocations': ['s3://iceberg']}}]}

In [ ]:
headers = {
"Authorization": f"Bearer {get_principal_token()}",
"Content-Type": "application/json"
}

response = requests.get(
    f"{POLARIS_URL}/api/management/v1/principal-roles/pyiceberg_service_account_role",
    headers=headers
)
response.json()